In [39]:
# 따릉이 대여소 현황 데이터 조회 (OPEN API)
# 조회된 데이터를 데이터베이스에 저장 (테이블 만들기 + 데이터 저장)

# apikey=5668556e4e6f686337344d77484770

In [40]:
# 라이브러리 준비
import requests
import json
import pandas as pd
import pymysql

In [41]:
# 데이터 요청 1
api_key="5668556e4e6f686337344d77484770"
url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/tbCycleStationInfo/1/5/"

resp = requests.get(url)
print(resp.content.decode('utf-8'))

{"stationInfo":{"list_total_count":"3143","RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다."},"row":[{"STA_LOC":"마포구","RENT_ID":"ST-10","RENT_NO":"00108","RENT_NM":"서교동 사거리","RENT_ID_NM":"108. 서교동 사거리","HOLD_NUM":"12","STA_ADD1":"서울특별시 마포구 양화로 93","STA_ADD2":"427","STA_LAT":"37.55274582","STA_LONG":"126.91861725","START_INDEX":0,"END_INDEX":0,"RNUM":"1"},{"STA_LOC":"광진구","RENT_ID":"ST-100","RENT_NO":"00503","RENT_NM":"더샵스타시티 C동 앞","RENT_ID_NM":"503. 더샵스타시티 C동 앞","HOLD_NUM":"15","STA_ADD1":"서울특별시 광진구 아차산로 262","STA_ADD2":"더샵스타시티 C동 앞","STA_LAT":"37.53666687","STA_LONG":"127.07359314","START_INDEX":0,"END_INDEX":0,"RNUM":"2"},{"STA_LOC":"양천구","RENT_ID":"ST-1000","RENT_NO":"00729","RENT_NM":"서부식자재마트 건너편","RENT_ID_NM":"729. 서부식자재마트 건너편","HOLD_NUM":"10","STA_ADD1":"서울특별시 양천구 신정동 236 ","STA_ADD2":"서부식자재마트 건너편","STA_LAT":"37.51037979","STA_LONG":"126.86679840","START_INDEX":0,"END_INDEX":0,"RNUM":"3"},{"STA_LOC":"양천구","RENT_ID":"ST-1002","RENT_NO":"00731","RENT_NM":"서울시 도로환경관리센터","RENT_ID_NM

In [42]:
# result_dict = json.loads(resp.content.decode('utf-8'))
result_dict = resp.json()
# result_dict
print( result_dict.keys() )
print( result_dict['stationInfo'].keys())
print( result_dict['stationInfo']['row'] )

dict_keys(['stationInfo'])
dict_keys(['list_total_count', 'RESULT', 'row'])
[{'STA_LOC': '마포구', 'RENT_ID': 'ST-10', 'RENT_NO': '00108', 'RENT_NM': '서교동 사거리', 'RENT_ID_NM': '108. 서교동 사거리', 'HOLD_NUM': '12', 'STA_ADD1': '서울특별시 마포구 양화로 93', 'STA_ADD2': '427', 'STA_LAT': '37.55274582', 'STA_LONG': '126.91861725', 'START_INDEX': 0, 'END_INDEX': 0, 'RNUM': '1'}, {'STA_LOC': '광진구', 'RENT_ID': 'ST-100', 'RENT_NO': '00503', 'RENT_NM': '더샵스타시티 C동 앞', 'RENT_ID_NM': '503. 더샵스타시티 C동 앞', 'HOLD_NUM': '15', 'STA_ADD1': '서울특별시 광진구 아차산로 262', 'STA_ADD2': '더샵스타시티 C동 앞', 'STA_LAT': '37.53666687', 'STA_LONG': '127.07359314', 'START_INDEX': 0, 'END_INDEX': 0, 'RNUM': '2'}, {'STA_LOC': '양천구', 'RENT_ID': 'ST-1000', 'RENT_NO': '00729', 'RENT_NM': '서부식자재마트 건너편', 'RENT_ID_NM': '729. 서부식자재마트 건너편', 'HOLD_NUM': '10', 'STA_ADD1': '서울특별시 양천구 신정동 236 ', 'STA_ADD2': '서부식자재마트 건너편', 'STA_LAT': '37.51037979', 'STA_LONG': '126.86679840', 'START_INDEX': 0, 'END_INDEX': 0, 'RNUM': '3'}, {'STA_LOC': '양천구', 'RENT_ID': 'ST-1002

In [43]:
# 데이터 요청 2
api_key="5668556e4e6f686337344d77484770"
url = "http://openapi.seoul.go.kr:8088/{0}/json/tbCycleStationInfo/{1}/{2}/"

all_stations = []
while True:
    start_idx = len(all_stations)
    resp = requests.get(url.format(api_key, start_idx + 1, start_idx + 1000))
    result = resp.json()
    all_stations += result['stationInfo']['row'] # 기존 데이터 + 새로 조회된 데이터
    total_cnt = int(result['stationInfo']['list_total_count'])
    if total_cnt <= len(all_stations): # 현재까지 조회한 데이터 갯수가 총 데이터 갯수와 같거나 크다면
        break

print("데이터 조회 완료 - 총 데이터 갯수 : ", len(all_stations))

데이터 조회 완료 - 총 데이터 갯수 :  3143


In [44]:
df_stations = pd.DataFrame(all_stations)
print( df_stations.shape )
df_stations.head(3)

(3143, 13)


,STA_LOC,RENT_ID,RENT_NO,RENT_NM,RENT_ID_NM,HOLD_NUM,STA_ADD1,STA_ADD2,STA_LAT,STA_LONG,START_INDEX,END_INDEX,RNUM
0,마포구,ST-10,00108,서교동 사거리,108. 서교동 사거리,12,서울특별시 마포구 양화로 93,427,37.55274582,126.91861725,0,0,1
1,광진구,ST-100,00503,더샵스타시티 C동 앞,503. 더샵스타시티 C동 앞,15,서울특별시 광진구 아차산로 262,더샵스타시티 C동 앞,37.53666687,127.07359314,0,0,2
2,양천구,ST-1000,00729,서부식자재마트 건너편,729. 서부식자재마트 건너편,10,서울특별시 양천구 신정동 236,서부식자재마트 건너편,37.51037979,126.86679840,0,0,3


In [45]:
df_stations.dtypes

STA_LOC        object
RENT_ID        object
RENT_NO        object
RENT_NM        object
RENT_ID_NM     object
HOLD_NUM       object
STA_ADD1       object
STA_ADD2       object
STA_LAT        object
STA_LONG       object
START_INDEX     int64
END_INDEX       int64
RNUM           object
dtype: object

In [46]:
df_stations.isnull().sum() # 실행 결과 HOLD_NUM 컬럼에 null 발견

STA_LOC         0
RENT_ID         0
RENT_NO         0
RENT_NM         0
RENT_ID_NM      0
HOLD_NUM       13
STA_ADD1        0
STA_ADD2        0
STA_LAT         0
STA_LONG        0
START_INDEX     0
END_INDEX       0
RNUM            0
dtype: int64

In [47]:
(df_stations['HOLD_NUM'].str.len() == 0).sum() # 2개의 행에 빈문자열('')이 있는 것을 확인 --> missing value로 해석

np.int64(2)

In [48]:
# 'HOLD_NUM' 컬럼의 missing value 처리

df_stations['HOLD_NUM'] = df_stations['HOLD_NUM'].fillna('0') # missing value를 특정한 값으로 변경
empty_str_mask = df_stations['HOLD_NUM'].str.len() == 0
# df_stations[empty_str_mask] = '0' # 빈문자열은 missing value로 해석해서 '0'으로 변경
df_stations.loc[empty_str_mask, 'HOLD_NUM'] = '0' # 빈문자열은 missing value로 해석해서 '0'으로 변경
df_stations['HOLD_NUM'] = df_stations['HOLD_NUM'].astype(int) # 자료형을 정수로 변경

print( df_stations['HOLD_NUM'].dtype, df_stations['HOLD_NUM'].isna().sum() )

int64 0


In [50]:
# 데이터를 저장할 테이블 만들기

try:
    # 데이터베이스 연결
    conn = pymysql.connect(
        host="127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    # 명령 실행기 준비
    cursor = conn.cursor()

    sql = "drop table if exists station"
    cursor.execute(sql)
    sql = """create table if not exists station
          (
            idx int primary key auto_increment,
            STA_LOC varchar(20) not null,
            RENT_ID varchar(20) not null,
            RENT_NO varchar(20) not null,
            RENT_NM varchar(50) not null,
            RENT_ID_NM varchar(50) not null,
            HOLD_NUM int not null,
            STA_ADD1 varchar(100) not null,
            STA_ADD2 varchar(100) null,
            STA_LAT varchar(15) not null,
            STA_LONG varchar(15) not null
          )"""
    cursor.execute(sql)    
except Exception as e:
    print("오류 발생 : ", e)
finally:
    # 연결 종료
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [51]:
# Open API로부터 가져온 데이터를 위에서 만든 테이블에 저장 

try:
    # 데이터베이스 연결
    conn = pymysql.connect(
        host="127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    # 명령 실행기 준비
    cursor = conn.cursor()

    # 명령 실행
    sql = "delete from station"
    cursor.execute(sql)
    sql = """insert into station 
             (STA_LOC, RENT_ID, RENT_NO, RENT_NM, RENT_ID_NM, HOLD_NUM,STA_ADD1, STA_ADD2, STA_LAT, STA_LONG)
             values 
             (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    data = df_stations.values[:, :-3].tolist() # Open API에서 가져온 데이터 중 마지막 3개 컬럼을 사용하지 않음
    cursor.executemany(sql, data)

    conn.commit() # 마지막 commit or rollback 이후에 실행된 모든 변경사항 확정
except Exception as e:
    print("오류 발생 : ", e)
    if conn:
        conn.rollback() # 마지막 commit or rollback 이후에 실행된 모든 변경사항 취소
finally:
    # 연결 종료
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [52]:
# 자치구별 대여소 갯수 데이터를 조회해서 파일로 저장

try:
    # 데이터베이스 연결
    conn = pymysql.connect(
        host="127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    # 명령 실행기 준비
    cursor = conn.cursor()

    # 명령 실행
    sql = """select sta_loc, count(*) cnt from station group by sta_loc"""
    cursor.execute(sql)

    station_data = cursor.fetchall() # 조회 결과를 tuple 형식으로 반환

    # 파일로 저장
    df_station = pd.DataFrame(station_data, 
                              columns=['자치구', '대여소갯수'])
    
    df_station.to_csv('data-files/station-count-by-gu.csv', index=False) # index 저장하지 않도록 설정
    
except Exception as e:
    print("오류 발생 : ", e)

finally:
    # 연결 종료
    if cursor:
        cursor.close()
    if conn:
        conn.close()